In [1]:
!pip install -q transformers datasets trl peft accelerate bitsandbytes sentencepiece

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.5/465.5 kB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 13.9 MB/s eta 0:00:00


In [2]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments
from trl import SFTTrainer
from peft import LoraConfig
import torch


model_name = "gpt2"

tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [3]:
from google.colab import files
uploaded = files.upload()

dataset = load_dataset("json", data_files="/content/novels_cleaned.jsonl", split="train")


print(dataset[0])

Saving novels_cleaned.jsonl to novels_cleaned.jsonl


Generating train split: 0 examples [00:00, ? examples/s]

{'book_id': '5a59b9cb5cb031ace0bff5a8957dd42d401e8809', 'chunk_id': '606f94f8-2e7c-4987-82d4-f2083ef703f7', 'title': 'Algernon-Blackwood John-Silence-Stories', 'chunk_index': 0, 'text': 'Case I A Psychical Invasion I “And what is it makes you think I could be of use in this particular case?” asked Dr. John Silence, looking across somewhat sceptically at the Swedish lady in the chair facing him. “Your sympathetic heart and your knowledge of occultism\ufeff—” “Oh, please\ufeff—that dreadful word!” he interrupted, holding up a finger with a gesture of impatience. “Well, then,” she laughed, “your wonderful clairvoyant gift and your trained psychic knowledge of the processes by which a personality may be disintegrated and destroyed\ufeff—these strange studies you’ve been experimenting with all these years\ufeff—” “If it’s only a case of multiple personality I must really cry off,” interrupted the doctor again hastily, a bored expression in his eyes. “It’s not that; now, please, be serious, 

In [4]:
from peft import LoraConfig

peft_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["c_attn"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)

In [5]:
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=1,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    learning_rate=2e-5,
    fp16=True,
    logging_steps=50,
    save_steps=200,
    save_total_limit=2,
    report_to="none",   # disable W&B or HF logging
)


In [6]:
def tokenize_function(example):
    return tokenizer(example["text"], truncation=True, max_length=512)

tokenized_dataset = dataset.map(tokenize_function, batched=True, remove_columns=dataset.column_names)

print(tokenized_dataset[0])

Map:   0%|          | 0/5993 [00:00<?, ? examples/s]

{'input_ids': [20448, 314, 317, 7562, 605, 25484, 314, 564, 250, 1870, 644, 318, 340, 1838, 345, 892, 314, 714, 307, 286, 779, 287, 428, 1948, 1339, 30, 447, 251, 1965, 1583, 13, 1757, 26354, 11, 2045, 1973, 6454, 30186, 1146, 379, 262, 14023, 10846, 287, 262, 5118, 6476, 683, 13, 564, 250, 7120, 22335, 2612, 290, 534, 3725, 286, 31587, 1042, 171, 119, 123, 960, 447, 251, 564, 250, 5812, 11, 3387, 171, 119, 123, 960, 5562, 35483, 1573, 0, 447, 251, 339, 19072, 11, 4769, 510, 257, 7660, 351, 257, 18342, 286, 28189, 1240, 13, 564, 250, 5779, 11, 788, 11, 447, 251, 673, 13818, 11, 564, 250, 14108, 7932, 537, 958, 40024, 415, 6979, 290, 534, 8776, 25058, 3725, 286, 262, 7767, 416, 543, 257, 8806, 743, 307, 29930, 4111, 290, 6572, 171, 119, 123, 960, 27218, 6283, 3640, 345, 447, 247, 303, 587, 27826, 351, 477, 777, 812, 171, 119, 123, 960, 447, 251, 564, 250, 1532, 340, 447, 247, 82, 691, 257, 1339, 286, 3294, 8806, 314, 1276, 1107, 3960, 572, 11, 447, 251, 19072, 262, 6253, 757, 37205, 11,

In [7]:
from trl import SFTTrainer, SFTConfig
from peft import LoraConfig


peft_config = LoraConfig(
    r=4,
    lora_alpha=16,
    target_modules=["c_attn"],
    lora_dropout=0.1,
    bias="none",
    task_type="CAUSAL_LM",
)


sft_config = SFTConfig(
    output_dir="./results",
    num_train_epochs=1,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=1,
    learning_rate=3e-5,
    fp16=False,
    bf16=False,
    logging_steps=50,
    save_steps=1000,
    save_total_limit=1,
    report_to="none",
)


In [8]:
trainer = SFTTrainer(
    model=AutoModelForCausalLM.from_pretrained(model_name),
    train_dataset=tokenized_dataset,
    peft_config=peft_config,
    args=sft_config,
)

trainer.train()

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

/usr/local/lib/python3.12/dist-packages/peft/tuners/lora/layer.py:2174: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


Truncating train dataset:   0%|          | 0/5993 [00:00<?, ? examples/s]

`loss_type=None` was set in the config but it is unrecognized. Using the default loss: `ForCausalLMLoss`.


Step,Training Loss
50,4.185500
100,4.211900
150,4.132200
200,4.053400
250,4.149500
300,4.114800
350,4.035400
400,4.147400
450,4.056600
500,4.216800


TrainOutput(global_step=5993, training_loss=3.956387307714306, metrics={'train_runtime': 422.248, 'train_samples_per_second': 14.193, 'train_steps_per_second': 14.193, 'total_flos': 810709179770880.0, 'train_loss': 3.956387307714306, 'entropy': 3.9348650699438052, 'num_tokens': 1585830.0, 'mean_token_accuracy': 0.30226876188156215, 'epoch': 1.0})

In [11]:
# Save adapter
trainer.model.save_pretrained("./finetuned_gpt2_epic")
tokenizer.save_pretrained("./finetuned_gpt2_epic")

print("Model saved to ./finetuned_gpt2_epic")

from transformers import pipeline
from peft import PeftModel

base_model = AutoModelForCausalLM.from_pretrained("gpt2")
model = PeftModel.from_pretrained(base_model, "./finetuned_gpt2_epic")

pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, device=0)

Model saved to ./finetuned_gpt2_epic


Device set to use cuda:0


In [10]:
prompt = "The sun dipped below the horizon and the knight"
result = pipe(prompt, max_new_tokens=80, temperature=0.8, top_p=0.9)

print("\nGenerated Text:\n")
print(result[0]["generated_text"])


Generated Text:

The sun dipped below the horizon and the knight was lying on his back, gazing up at the beautiful face of the young maiden. A woman had been standing by and greeting him, and the knight was sitting down, watching her closely. The woman stood in front of him, and his mother, wearing a silken gown, spoke. ”What do you want?” She said. “I want you to be my wife.
